# 混合检索（Vector embdedding + BM25）

混合搜索结合了不同搜索范式的优势，以提高检索的准确性和鲁棒性。它既能利用密集向量搜索和稀疏向量搜索的能力，也能利用多种密集向量搜索策略的组合，确保对各种查询进行全面而精确的检索。

**混合检索的组成与原理** 
- 向量检索（语义匹配）    
通过将文本转化为高维向量，计算语义相似度召回相关内容。  
优势包括：  
    - 语义理解：支持复杂语义匹配（如“猫追老鼠”与“猫捕猎老鼠”）  
    - 多模态与跨语言：可处理文本、图像、音视频，并支持跨语言检索（如中文查英文内容）  
    - 容错性：对拼写错误、模糊描述有较强鲁棒性    
局限性：难以精准匹配专有名词（如“iPhone 15”）、缩写（如“RAG”）或ID等短文本。  
- 关键词检索（精确匹配）    
基于关键词倒排索引或BM25算法，  
优势包括：  
    - 精准匹配：适合人名、产品名、代码片段等精确查询  
    - 低频词捕捉：能识别关键低频词（如“喝咖啡”中的“咖啡”）  
局限性：缺乏语义理解能力（例如“北大医院”无法关联“北京大学第一医院”）。  

**混合检索的优势**
- 覆盖全场景需求  
既能处理模糊语义（如用户提问“如何解决失眠”），也能精准匹配特定实体（如“GPT-4的发布时间”）。  
- 提升召回率与准确性  
实验表明，混合检索相较单一方法可使文档召回率提升20%-30%，尤其适合复杂查询（如含实体与语义混合的提问）。  
- 降低噪声干扰  
通过重排序技术（如元数据过滤、上下文重排）筛选冗余信息，减少大模型生成时的幻觉风险。  



# Mlivus的混合检索+Rerank

https://milvus.io/docs/zh/milvus_hybrid_search_retriever.md

![Milvus](../../assets/rag9.png)

本图展示了最常见的混合搜索方案，即密集+稀疏混合搜索。在这种情况下，使用语义向量相似性和精确关键词匹配两种方法检索候选内容。来自这些方法的结果会被合并、重新排序，并传递给 LLM 以生成最终答案。这种方法兼顾了精确性和语义理解，对各种查询场景都非常有效。  

除了密集+稀疏混合搜索，混合策略还可以结合多个密集向量模型。例如，一种密集向量模型可能专门捕捉语义的细微差别，而另一种则侧重于上下文嵌入或特定领域的表示。通过合并这些模型的结果并重新排序，这种类型的混合搜索可确保检索过程更加细致入微、更能感知上下文。  

LangChain Milvus集成提供了实现混合搜索的灵活方式，它支持任意数量的向量场，以及任意自定义的密集或稀疏嵌入模型，这使得LangChain Milvus能够灵活适应各种混合搜索使用场景，同时兼容LangChain的其他功能。

# 以下内容方便更好理解

## 混合检索实现

### 固定权重

获取text用于BM25检索

In [ ]:
from pymilvus import connections, Collection


def get_text_list_from_milvus(
        collection_name: str,
        host: str = "192.168.0.188",
        port: str = "19530",
        expr: str = "",
        limit: int = 1000,
        output_fields: list = ["text"],
) -> list:
    """
    从 Milvus 集合中读取指定字段（默认是 text）并返回列表

    Args:
        collection_name: Milvus 集合名称
        host: Milvus 服务器地址（默认 localhost）
        port: Milvus 端口（默认 19530）
        expr: 过滤条件表达式（默认无过滤）
        limit: 返回数据条数上限（默认 1000）
        output_fields: 要提取的字段列表（默认 ["text"]）

    Returns:
        list: 包含目标字段值的列表
    """
    # 1. 连接 Milvus
    connections.connect(alias="default", host=host, port=port)

    # 2. 加载集合
    collection = Collection(name=collection_name)
    collection.load()

    # 3. 查询数据
    results = collection.query(
            expr=expr,
            output_fields=output_fields,
            limit=limit
        )

    # 4. 提取目标字段为列表
    if not output_fields:
        raise ValueError("output_fields 不能为空")

    field_name = output_fields[0]  # 默认取第一个字段
    data_list = [item[field_name] for item in results]
    return data_list


# 示例调用
if __name__ == "__main__":
    # 示例1：读取默认的 text 字段
    texts = get_text_list_from_milvus(collection_name="Vmaxs")
    print(f"获取 {len(texts)} 条文本，前5条: {texts[:5]}")

In [ ]:
from langchain_chroma import Chroma
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.retrievers import BM25Retriever
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from pymilvus import Collection, connections



# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )
    return vectordb

def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

def get_text_list_from_milvus(
        collection_name: str,
        host: str = "192.168.0.188",
        port: str = "19530",
        expr: str = "",
        limit: int = 1000,
        output_fields: list = ["text"],
) -> list:
    """
    从 Milvus 集合中读取指定字段（默认是 text）并返回列表
    """
    # 1. 连接 Milvus
    connections.connect(alias="default", host=host, port=port)

    # 2. 加载集合
    collection = Collection(name=collection_name)
    collection.load()

    # 3. 查询数据
    results = collection.query(
            expr=expr,
            output_fields=output_fields,
            limit=limit
        )

    # 4. 提取目标字段为列表
    if not output_fields:
        raise ValueError("output_fields 不能为空")

    field_name = output_fields[0]  # 默认取第一个字段
    data_list = [item[field_name] for item in results]
    return data_list

def get_qa_chain_with_memory(question: str):
    # Initialize memory outside the function so it persists across questions
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    
    vectordb = get_vectordb()

    # 1. 初始化 BM25 检索器（关键词检索）
    documents = get_text_list_from_milvus(collection_name="Vmaxs")
    bm25_retriever = BM25Retriever.from_texts(documents)
    bm25_retriever.k = 10  # 返回前10个BM25检索结果

    # 2. 初始化向量检索器
    vector_retriever = vectordb.as_retriever(
        search_kwargs={"k": 10},  # 返回前10个向量检索结果
        search_type="mmr",  # 多样性检索
    )

    # 3. 混合检索（EnsembleRetriever）作为最终检索器
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[0.5, 0.5],  # 调整BM25和向量检索的权重
    )

    # 4. 定义提示模板
    # 修改后的Prompt模板（添加chat_history变量）
    template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
    QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
    # 5. 构建对话式检索链
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=ensemble_retriever,  # 直接使用混合检索器
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_PROMPT
        },
        verbose=False
    )

    result = qa_chain({"question": question})
    return result

# 测试问题
questions = [
    "什么是VMAX？",
    "VMAX有哪些功能？",
    "整理成excel表格"
]

for question in questions:
    result = get_qa_chain_with_memory(question)
    print("\n" + "=" * 50 + "\n")

### 动态权重

In [ ]:
from langchain_chroma import Chroma
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.retrievers import BM25Retriever
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from pymilvus import Collection, connections
import re

# Initialize memory outside the function so it persists across questions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )
    return vectordb

def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

def get_text_list_from_milvus(
        collection_name: str,
        host: str = "192.168.0.188",
        port: str = "19530",
        expr: str = "",
        limit: int = 1000,
        output_fields: list = ["text"],
) -> list:
    """
    从 Milvus 集合中读取指定字段（默认是 text）并返回列表
    """
    # 1. 连接 Milvus
    connections.connect(alias="default", host=host, port=port)

    # 2. 加载集合
    collection = Collection(name=collection_name)
    collection.load()

    # 3. 查询数据
    results = collection.query(
            expr=expr,
            output_fields=output_fields,
            limit=limit
        )

    # 4. 提取目标字段为列表
    if not output_fields:
        raise ValueError("output_fields 不能为空")

    field_name = output_fields[0]  # 默认取第一个字段
    data_list = [item[field_name] for item in results]
    return data_list

def determine_query_type(question: str) -> str:
    """
    根据问题内容判断查询类型，返回权重调整策略

    返回:
        "keyword" - 更适合关键词检索的问题
        "semantic" - 更适合语义检索的问题
        "balanced" - 平衡型问题
    """
    # 关键词型问题特征
    keyword_patterns = [
        r"什么是.*\?",  # 定义类问题
        r".*包括哪些.*",  # 列举类问题
        r".*有哪些.*",  # 列举类问题
        r".*多少种.*",  # 数量类问题
        r".*步骤.*",  # 流程类问题
        r".*如何.*",  # 方法类问题
        r".*怎样.*",  # 方法类问题
        r".*整理.*表格",  # 结构化输出要求
        r".*列出.*",  # 列举要求
        r".*对比.*",  # 比较类问题
    ]

    # 语义型问题特征
    semantic_patterns = [
        r".*解决.*问题",  # 解决方案类
        r".*原因.*",  # 原因分析类
        r".*为什么.*",  # 原因分析类
        r".*建议.*",  # 建议类
        r".*优缺点.*",  # 分析类
        r".*影响.*",  # 影响分析类
        r".*解释.*",  # 解释说明类
        r".*理解.*",  # 理解类
        r".*意味着什么",  # 含义类
    ]

    # 检查是否是关键词型问题
    for pattern in keyword_patterns:
        if re.search(pattern, question):
            return "keyword"

    # 检查是否是语义型问题
    for pattern in semantic_patterns:
        if re.search(pattern, question):
            return "semantic"

    # 默认平衡型
    return "balanced"


def get_dynamic_weights(query_type: str) -> tuple:
    """
    根据查询类型返回动态权重

    返回:
        tuple: (bm25_weight, vector_weight)
    """
    if query_type == "keyword":
        return (0.7, 0.3)  # 更侧重关键词检索
    elif query_type == "semantic":
        return (0.3, 0.7)  # 更侧语义检索
    else:
        return (0.5, 0.5)  # 平衡权重

def get_qa_chain_with_memory(question: str):
    vectordb = get_vectordb()

    # 1. 确定查询类型和动态权重
    query_type = determine_query_type(question)
    bm25_weight, vector_weight = get_dynamic_weights(query_type)
    print(f"问题类型: {query_type}, 权重设置: BM25={bm25_weight}, Vector={vector_weight}")

    # 2. 初始化 BM25 检索器
    documents = get_text_list_from_milvus(collection_name="Vmaxs")
    bm25_retriever = BM25Retriever.from_texts(documents)
    bm25_retriever.k = 10

    # 3. 初始化向量检索器
    vector_retriever = vectordb.as_retriever(
        search_kwargs={"k": 10},
        search_type="mmr",
    )

    # 4. 使用动态权重的混合检索
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[bm25_weight, vector_weight],  # 使用动态权重
    )
    # 5. 混合检索（EnsembleRetriever）作为最终检索器
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[0.5, 0.5],  # 调整BM25和向量检索的权重
    )

   # 6. 定义提示模板（加入权重信息）
   # 4. 定义提示模板
    # 修改后的Prompt模板（添加chat_history变量）
    template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
    QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
    # 5. 构建对话式检索链
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=ensemble_retriever,  # 直接使用混合检索器
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_PROMPT
        },
        verbose=False
    )

    result = qa_chain({"question": question})
    return result

# 测试问题
questions = [
    "什么是VMAX的上网日志业务？",
    "上网日志业务包含哪些功能？"
]

for question in questions:
    print(f"\n问题: {question}")
    result = get_qa_chain_with_memory(question)
    print(f"\n回答: {result['answer']}")
    print("=" * 50)

## 密集嵌入 + Milvus BM25 内置功能

https://milvus.io/docs/zh/milvus_hybrid_search_retriever.md

## Rerank

混合检索结合了向量搜索、关键词匹配等多路径检索方法，能覆盖更广泛的候选文档（高召回率）。但不同检索算法的结果可能存在冗余或噪声，例如：  
- 向量搜索因信息压缩（如768维嵌入）导致细节丢失，可能将关键文档排在靠后位置；  
- 关键词检索容易受语义鸿沟影响，检索结果可能包含字面匹配但语义无关的内容。  

此时，Rerank通过交叉编码器（Cross-Encoder）对混合检索结果进行实时深度语义分析，直接计算查询与文档的匹配度，过滤低相关性内容。例如，原本混合检索返回的10篇文档中，真正相关的可能仅有3篇，Rerank可将其精准筛选并排序，使LLM获得高纯度上下文

#### cohere

In [ ]:
from langchain.retrievers.document_compressors import CohereRerank
# 从环境变量中加载你的 API_KEY
_ = load_dotenv(find_dotenv())    # read local .env file
cohere_api_key = os.environ['COHERE_API_KEY']

# rerank检索
# Cohere Rerank配置

import cohere
cohere_client = cohere.Client(api_key="cohere_api_key")

compressor = CohereRerank(
    client=cohere_client,
    top_n=5,
    model="rerank-multilingual-v3.0"  # 支持多语言的新版本
)


#### jina

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank  # 使用Jina的rerank组件
# 从环境变量中加载你的 API_KEY
_ = load_dotenv(find_dotenv())    # read local .env file
jina_api_key = os.environ['JINA_API_KEY']

compressor = JinaRerank(
    jina_api_key=jina_api_key,
    top_n=3,
    model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
)

#### BGE版本

In [ ]:
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
# from langchain_community.document_compressors import JinaRerank  # 使用Jina的rerank组件

# BGE配置
# 先将模型下载到本地
# modelscope download --model BAAI/bge-reranker-base --cache_dir /opt/workspace/models

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="/opt/workspace/models/BAAI/bge-reranker-base")
compressor = CrossEncoderReranker(model=model, top_n=3)

# 混合检索 + rerank

从本地获取BM25检索的DOC

In [ ]:
from langchain_chroma import Chroma
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.retrievers import BM25Retriever
import cohere

# Initialize memory outside the function so it persists across questions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )
    return vectordb

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:14b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

def get_qa_chain_with_memory(question: str):
    vectordb = get_vectordb()

    # 1. 初始化 BM25 检索器（关键词检索）
    # 假设你已经有一个文档列表 `documents`（如果没有，可以从 vectordb 获取）
    # 示例：documents = vectordb.get_all_documents()
    # 这里仅作演示，实际使用时需要替换成你的文档数据
    documents = ["doc1", "doc2", "doc3"]  # 替换成你的文档
    # 从 Milvus 获取所有原始文本
    # documents = get_documents_from_milvus()
    bm25_retriever = BM25Retriever.from_texts(documents)
    bm25_retriever.k = 10  # 返回前10个BM25检索结果

    # 2. 初始化向量检索器
    vector_retriever = vectordb.as_retriever(
        search_kwargs={"k": 10},  # 返回前10个向量检索结果
        search_type="mmr",  # 多样性检索
    )

    # 3. 混合检索（EnsembleRetriever）
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[0.5, 0.5],  # 调整BM25和向量检索的权重
    )

    # 4. 使用 Jina Rerank 优化结果
    # Jina Rerank配置
    JINA_API_KEY = "jina_63bb115e2d5f42d581f42643294792b5CE4nrEINMDcT4vJZJaSLcr5tkbIB"  # 替换为你的Jina API密钥
    
    compressor = JinaRerank(
        jina_api_key=JINA_API_KEY,
        top_n=3,
        model="jina-reranker-v2-base-multilingual"  # Jina的多语言rerank模型[5](@ref)
    )


    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=ensemble_retriever  # 使用混合检索作为基础检索器
    )

    # 5. 定义提示模板
    # 4. 定义提示模板
    # 修改后的Prompt模板（添加chat_history变量）
    template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
    QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
    # 5. 构建对话式检索链
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=ensemble_retriever,  # 直接使用混合检索器
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_PROMPT
        },
        verbose=False
    )

    result = qa_chain({"question": question})
    return result


# 测试问题
questions = [
    "什么是VMAX的上网日志业务？",
    "上网日志业务包含哪些功能？",
    "整理成excel表格"
]

for question in questions:
    result = get_qa_chain_with_memory(question)
    # print(f"问题：{question}")
    # print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "=" * 50 + "\n")


获取milvus数据库中数据text字段

In [ ]:
from pymilvus import connections, Collection


def get_text_list_from_milvus(
        collection_name: str,
        host: str = "129.201.70.35",
        port: str = "19530",
        expr: str = "",
        limit: int = 1000,
        output_fields: list = ["text"],
) -> list:
    """
    从 Milvus 集合中读取指定字段（默认是 text）并返回列表

    Args:
        collection_name: Milvus 集合名称
        host: Milvus 服务器地址（默认 localhost）
        port: Milvus 端口（默认 19530）
        expr: 过滤条件表达式（默认无过滤）
        limit: 返回数据条数上限（默认 1000）
        output_fields: 要提取的字段列表（默认 ["text"]）

    Returns:
        list: 包含目标字段值的列表
    """
    # 1. 连接 Milvus
    connections.connect(alias="default", host=host, port=port)

    # 2. 加载集合
    collection = Collection(name=collection_name)
    collection.load()

    # 3. 查询数据
    results = collection.query(
            expr=expr,
            output_fields=output_fields,
            limit=limit
        )

    # 4. 提取目标字段为列表
    if not output_fields:
        raise ValueError("output_fields 不能为空")

    field_name = output_fields[0]  # 默认取第一个字段
    data_list = [item[field_name] for item in results]
    return data_list


# 示例调用
if __name__ == "__main__":
    # 示例1：读取默认的 text 字段
    texts = get_text_list_from_milvus(collection_name="Vmaxs")
    print(f"获取 {len(texts)} 条文本，前5条: {texts[:5]}")

将本地DOC替换

In [ ]:
from langchain_chroma import Chroma
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.retrievers import BM25Retriever
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import cohere

# Initialize memory outside the function so it persists across questions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )
    return vectordb


def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1, streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])


from pymilvus import Collection

def get_text_list_from_milvus(
        collection_name: str,
        host: str = "192.168.0.188",
        port: str = "19530",
        expr: str = "",
        limit: int = 1000,
        output_fields: list = ["text"],
) -> list:
    """
    从 Milvus 集合中读取指定字段（默认是 text）并返回列表

    Args:
        collection_name: Milvus 集合名称
        host: Milvus 服务器地址（默认 localhost）
        port: Milvus 端口（默认 19530）
        expr: 过滤条件表达式（默认无过滤）
        limit: 返回数据条数上限（默认 1000）
        output_fields: 要提取的字段列表（默认 ["text"]）

    Returns:
        list: 包含目标字段值的列表
    """
    # 1. 连接 Milvus
    connections.connect(alias="default", host=host, port=port)

    # 2. 加载集合
    collection = Collection(name=collection_name)
    collection.load()

    # 3. 查询数据
    results = collection.query(
            expr=expr,
            output_fields=output_fields,
            limit=limit
        )

    # 4. 提取目标字段为列表
    if not output_fields:
        raise ValueError("output_fields 不能为空")

    field_name = output_fields[0]  # 默认取第一个字段
    data_list = [item[field_name] for item in results]
    return data_list


def get_qa_chain_with_memory(question: str):
    vectordb = get_vectordb()

    # 1. 初始化 BM25 检索器（关键词检索）
    # 假设你已经有一个文档列表 `documents`（如果没有，可以从 vectordb 获取）
    # 示例：documents = vectordb.get_all_documents()
    # 这里仅作演示，实际使用时需要替换成你的文档数据
    # documents = ["doc1", "doc2", "doc3"]  # 替换成你的文档
    # 从 Milvus 获取所有原始文本
    documents = get_text_list_from_milvus(collection_name="Vmaxs")
    bm25_retriever = BM25Retriever.from_texts(documents)
    bm25_retriever.k = 10  # 返回前10个BM25检索结果

    # 2. 初始化向量检索器
    vector_retriever = vectordb.as_retriever(
        search_kwargs={"k": 10},  # 返回前10个向量检索结果
        search_type="mmr",  # 多样性检索
    )

    # 3. 混合检索（EnsembleRetriever）
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[0.5, 0.5],  # 调整BM25和向量检索的权重
    )

    # 4. 使用 Cohere Rerank 优化结果
    cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
    compressor = CohereRerank(
        client=cohere_client,
        top_n=5,  # 最终保留5个最相关的文档
        model="rerank-multilingual-v3.0"
    )

    compression_retriever = ContextualCompressionRetriever(
        base_retriever=ensemble_retriever,  # 使用混合检索作为基础检索器
        base_compressor=compressor # 进行rerank

    )

    # 5. 定义提示模板
    # 4. 定义提示模板
    # 修改后的Prompt模板（添加chat_history变量）
    template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
    QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
    # 5. 构建对话式检索链
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=ensemble_retriever,  # 直接使用混合检索器
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_PROMPT
        },
        verbose=False
    )

    result = qa_chain({"question": question})
    return result


# 测试问题
questions = [
    "什么是VMAX的上网日志业务？",
    "上网日志业务包含哪些功能？",
    "整理成excel表格"
]

for question in questions:
    result = get_qa_chain_with_memory(question)
    # print(f"问题：{question}")
    # print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "=" * 50 + "\n")


# 混合检索动态权重 + rerank

In [ ]:
from langchain_chroma import Chroma
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever, EnsembleRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.retrievers import BM25Retriever
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import cohere
from pymilvus import connections, Collection
import re

# Initialize memory outside the function so it persists across questions
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


# 初始化 Milvus 向量数据库
def get_vectordb():
    emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

    # Milvus 连接参数
    vectordb = Milvus(
        embedding_function=emb_bgem3,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )
    return vectordb


def get_llm():
    return OllamaLLM(base_url='http://localhost:11434', model='deepseek-r1:1.5b', temperature=0.1, streaming=True,
                     callbacks=[StreamingStdOutCallbackHandler()])


def get_text_list_from_milvus(
        collection_name: str,
        host: str = "192.168.0.188",
        port: str = "19530",
        expr: str = "",
        limit: int = 1000,
        output_fields: list = ["text"],
) -> list:
    """
    从 Milvus 集合中读取指定字段（默认是 text）并返回列表
    """
    connections.connect(alias="default", host=host, port=port)
    collection = Collection(name=collection_name)
    collection.load()
    results = collection.query(
        expr=expr,
        output_fields=output_fields,
        limit=limit
    )
    field_name = output_fields[0]
    data_list = [item[field_name] for item in results]
    return data_list


def determine_query_type(question: str) -> str:
    """
    根据问题内容判断查询类型，返回权重调整策略

    返回:
        "keyword" - 更适合关键词检索的问题
        "semantic" - 更适合语义检索的问题
        "balanced" - 平衡型问题
    """
    # 关键词型问题特征
    keyword_patterns = [
        r"什么是.*\?",  # 定义类问题
        r".*包括哪些.*",  # 列举类问题
        r".*有哪些.*",  # 列举类问题
        r".*多少种.*",  # 数量类问题
        r".*步骤.*",  # 流程类问题
        r".*如何.*",  # 方法类问题
        r".*怎样.*",  # 方法类问题
        r".*整理.*表格",  # 结构化输出要求
        r".*列出.*",  # 列举要求
        r".*对比.*",  # 比较类问题
    ]

    # 语义型问题特征
    semantic_patterns = [
        r".*解决.*问题",  # 解决方案类
        r".*原因.*",  # 原因分析类
        r".*为什么.*",  # 原因分析类
        r".*建议.*",  # 建议类
        r".*优缺点.*",  # 分析类
        r".*影响.*",  # 影响分析类
        r".*解释.*",  # 解释说明类
        r".*理解.*",  # 理解类
        r".*意味着什么",  # 含义类
    ]

    # 检查是否是关键词型问题
    for pattern in keyword_patterns:
        if re.search(pattern, question):
            return "keyword"

    # 检查是否是语义型问题
    for pattern in semantic_patterns:
        if re.search(pattern, question):
            return "semantic"

    # 默认平衡型
    return "balanced"


def get_dynamic_weights(query_type: str) -> tuple:
    """
    根据查询类型返回动态权重

    返回:
        tuple: (bm25_weight, vector_weight)
    """
    if query_type == "keyword":
        return (0.7, 0.3)  # 更侧重关键词检索
    elif query_type == "semantic":
        return (0.3, 0.7)  # 更侧语义检索
    else:
        return (0.5, 0.5)  # 平衡权重


def get_qa_chain_with_memory(question: str):
    vectordb = get_vectordb()

    # 1. 确定查询类型和动态权重
    query_type = determine_query_type(question)
    bm25_weight, vector_weight = get_dynamic_weights(query_type)
    print(f"问题类型: {query_type}, 权重设置: BM25={bm25_weight}, Vector={vector_weight}")

    # 2. 初始化 BM25 检索器
    documents = get_text_list_from_milvus(collection_name="Vmaxs")
    bm25_retriever = BM25Retriever.from_texts(documents)
    bm25_retriever.k = 10

    # 3. 初始化向量检索器
    vector_retriever = vectordb.as_retriever(
        search_kwargs={"k": 10},
        search_type="mmr",
    )

    # 4. 使用动态权重的混合检索
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[bm25_weight, vector_weight],  # 使用动态权重
    )

    # 5. 使用 Cohere Rerank 优化结果
    cohere_client = cohere.Client(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ")
    compressor = CohereRerank(
        client=cohere_client,
        top_n=5,
        model="rerank-multilingual-v3.0"
    )

    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,
        base_retriever=ensemble_retriever
    )

    # 6. 定义提示模板（加入权重信息）
    # 4. 定义提示模板
    # 修改后的Prompt模板（添加chat_history变量）
    template = """你是VMAX运维助手，请参考以下对话历史和上下文来回答问题：
    {chat_history}
    
    相关上下文：
    {context}
    
    问题：{question}
    回答结束时说“谢谢你的提问！”
    """
    
    QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=template
    )
    
    # 5. 构建对话式检索链
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=get_llm(),
        retriever=ensemble_retriever,  # 直接使用混合检索器
        memory=memory,
        output_key="answer",
        combine_docs_chain_kwargs={
            "prompt": QA_PROMPT
        },
        verbose=False
    )

    result = qa_chain({"question": question})
    return result


# 测试问题
questions = [
    "什么是VMAX的上网日志业务？",  # 定义类问题，更适合关键词检索
    "上网日志业务包含哪些功能？",  # 列举类问题，更适合关键词检索
    "整理成excel表格",  # 结构化输出要求，更适合关键词检索
    "为什么我的VMAX设备会出现日志丢失问题？",  # 原因分析类，更适合语义检索
    "如何解决VMAX日志存储空间不足的问题？",  # 解决方案类，更适合语义检索
    "VMAX-S与其他型号的主要区别是什么？"  # 平衡型问题
]

for question in questions:
    print(f"\n问题: {question}")
    result = get_qa_chain_with_memory(question)
    print(f"\n回答: {result['answer']}")
    print("=" * 50)